# MINI PROJECT 1  - The TF-IDF Data Set

In [1]:
import nltk
import string
import gensim
import pickle 
import pyLDAvis
import math
import itertools
import unicodedata
import pyLDAvis.gensim_models

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import gensim.corpora as corpora

from __future__ import division
from pprint import pprint
from nltk.tokenize import  word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
STOPWORDS = set(stopwords.words('english'))

In [3]:
corpus_file = open('TF-IDF_dataset.txt','r',encoding='utf8')

In [4]:
corpus = corpus_file.read()

#### 1. Remove Stopwords

In [5]:
corpus = " ".join([word for word in corpus.split() if word not in STOPWORDS])

#### 2. Remove the punctuations. the special characters and convert the text to lower case.

In [6]:
corpus = ''.join(e for e in corpus if (e.isalnum() or e.isspace()))
corpus = ''.join([c for c in corpus if c not in string.punctuation])
corpus = corpus.lower()

In [7]:
# corpus
## uncomment the above line of code to print out the enitre corpus in a single sring

#### Dataset Preparation

In [8]:
with open('TF-IDF_dataset.txt','r',encoding='utf8') as file:
    all_chapters = file.read()

In [9]:
chapter_dict = {}

indexes = []
for i in range(1,9):
    indexes.append(f"Chapter {i}")

for i in range(0,8):
    if i == 7:
        str1 = indexes[i]
        start_chapter = all_chapters.find(str1)
        chapter_dict["chapter{0}".format(i+1)] = all_chapters[start_chapter+9:]

    else:
        str1 = indexes[i]
        str2 = indexes[i+1]
        start_chapter = all_chapters.find(str1)
        end_chapter = all_chapters.find(str2)
        if start_chapter == -1:
            start_chapter = all_chapters.find(str1.lower())
        if end_chapter == -1:
            end_chapter = all_chapters.find(str2.lower())

        chapter_dict["chapter{0}".format(i+1)] = all_chapters[start_chapter+9:end_chapter]

In [10]:
for i in range(0,8):
    this_key = list(chapter_dict.keys())[i]
    current_str = chapter_dict.get(this_key)
    current_str = " ".join([word for word in str(current_str).split() if word not in STOPWORDS])
    chapter_dict[this_key] = current_str

In [11]:
for i in range(0,8):
    this_key = list(chapter_dict.keys())[i]
    current_str = chapter_dict.get(this_key)
    current_str = ''.join(e for e in current_str if (e.isalnum() or e.isspace()))
    current_str = ''.join([c for c in current_str if c not in string.punctuation])
    chapter_dict[this_key] = current_str.lower()

#### 3. Create bigrams and trigrams for the entire dataset and list down 20 most frequent bigram and 20 most frequent trigrams 

In [12]:
corpus_word_list = corpus.split()

In [13]:
(pd.Series(nltk.ngrams(corpus_word_list, 2)).value_counts())[:20]

(i, found)               2
(natural, philosophy)    2
(ingolstadt, i)          2
(when, i)                2
(it, already)            2
(me, i)                  2
(thought, returning)     2
(m, waldman)             2
(return, us)             2
(my, father)             2
(long, time)             2
(native, country)        2
(two, years)             2
(i, thought)             2
(her, she)               2
(i, made)                2
(cause, death)           1
(mockery, justice)       1
(rapid, my)              1
(rowing, lake)           1
dtype: int64

In [14]:
(pd.Series(nltk.ngrams(corpus_word_list, 3)).value_counts())[:20]

(mountains, changes, seasons)       1
(cruel, kindness, i)                1
(clerval, son, merchant)            1
(cannot, rendered, callous)         1
(syndics, father, filled)           1
(rapidly, end, two)                 1
(roncesvalles, round, table)        1
(stars, often, disappeared)         1
(clear, facile, apprehension)       1
(father, rest, family)              1
(unfortunate, circumstances, he)    1
(shapes, mountains, changes)        1
(league, city, we)                  1
(eye, creature, open)               1
(friendship, one, among)            1
(eye, saw, us)                      1
(chapter, 3, when)                  1
(knightly, adventure, he)           1
(eye, skims, page)                  1
(persuade, mother, refrain)         1
dtype: int64

#### 4. You have to implement TF-IDF the Algorithm from scratch. 

In [15]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [16]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if val != 0:
            idfDict[word] = math.log10(N / float(val))
        else:
            idfDict[word] = math.log10(N / float(val+1))
        
    return idfDict

In [17]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

#### 5. Use the above-implemented algorithm and the values to calculate TF-IDF (using TF IDF formula) on the dataset and list down the top 10 words which have the highest TF-IDF Value.

In [18]:
word_set = [ ]
x = corpus.split(" ")
for word in x:
    if word not in word_set:
        word_set.append(word)

In [19]:
bow1 = chapter_dict.get('chapter1').split(" ")
bow2 = chapter_dict.get('chapter2').split(" ")
bow3 = chapter_dict.get('chapter3').split(" ")
bow4 = chapter_dict.get('chapter4').split(" ")
bow5 = chapter_dict.get('chapter5').split(" ")
bow6 = chapter_dict.get('chapter6').split(" ")
bow7 = chapter_dict.get('chapter7').split(" ")
bow8 = chapter_dict.get('chapter8').split(" ")

In [20]:
wordDict1 = dict.fromkeys(word_set, 0) 
wordDict2 = dict.fromkeys(word_set, 0)
wordDict3 = dict.fromkeys(word_set, 0) 
wordDict4 = dict.fromkeys(word_set, 0)
wordDict5 = dict.fromkeys(word_set, 0) 
wordDict6 = dict.fromkeys(word_set, 0)
wordDict7 = dict.fromkeys(word_set, 0) 
wordDict8 = dict.fromkeys(word_set, 0)

In [21]:
for word in bow1:
    wordDict1[word]+=1
    
for word in bow2:
    wordDict2[word]+=1
    
for word in bow3:
    wordDict3[word]+=1
    
for word in bow4:
    wordDict4[word]+=1

for word in bow5:
    wordDict5[word]+=1
    
for word in bow6:
    wordDict6[word]+=1

for word in bow7:
    wordDict7[word]+=1

for word in bow8:
    wordDict8[word]+=1

In [22]:
pd.DataFrame([wordDict1, wordDict2, wordDict3, wordDict4, wordDict5, wordDict6, wordDict7, wordDict8])

,chapter,1,i,birth,genevese,family,one,distinguished,republic,my,...,seated,tear,dim,recovered,herself,look,sorrowful,attest,utter,guiltlessness
0,0,0,2,1,1,2,2,2,1,2,...,0,0,0,0,0,0,0,0,0,0
1,0,0,7,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,8,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0,0,13,0,0,0,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,7,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,6,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,5,0,0,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1


In [23]:
tfBow1 = computeTF(wordDict1, bow1)
tfBow2 = computeTF(wordDict2, bow2)
tfBow3 = computeTF(wordDict3, bow3)
tfBow4 = computeTF(wordDict4, bow4)
tfBow5 = computeTF(wordDict5, bow5)
tfBow6 = computeTF(wordDict6, bow6)
tfBow7 = computeTF(wordDict7, bow7)
tfBow8 = computeTF(wordDict8, bow8)

In [24]:
idfs = computeIDF([wordDict1, wordDict2, wordDict3, wordDict4, wordDict5, wordDict6, wordDict7, wordDict8])

In [25]:
tfidfBow1 = computeTFIDF(tfBow1, idfs)
tfidfBow2 = computeTFIDF(tfBow2, idfs)
tfidfBow3 = computeTFIDF(tfBow3, idfs)
tfidfBow4 = computeTFIDF(tfBow4, idfs)
tfidfBow5 = computeTFIDF(tfBow5, idfs)
tfidfBow6 = computeTFIDF(tfBow6, idfs)
tfidfBow7 = computeTFIDF(tfBow7, idfs)
tfidfBow8 = computeTFIDF(tfBow8, idfs)

In [26]:
dataframe = pd.DataFrame([tfidfBow1, tfidfBow2, tfidfBow3, tfidfBow4, tfidfBow5, tfidfBow6, tfidfBow7, tfidfBow8])

In [27]:
dataframe.shape

(8, 945)

In [28]:
dataframe.mean(axis = 0).sort_values(ascending = False)[:10]

he               0.003679
would            0.002654
circumstances    0.002509
return           0.002136
father           0.002125
yellow           0.002016
limbs            0.002016
almost           0.002016
black            0.002016
you              0.002008
dtype: float64

#### 6. Label the cleaned Tf-IDF dataset

In [29]:
Tagged_list = nltk.pos_tag(corpus.split())

In [30]:
len(Tagged_list)

1381

In [31]:
print(Tagged_list[:20])

[('chapter', 'NN'), ('1', 'CD'), ('i', 'JJ'), ('birth', 'NN'), ('genevese', 'JJ'), ('family', 'NN'), ('one', 'CD'), ('distinguished', 'VBN'), ('republic', 'JJ'), ('my', 'PRP$'), ('ancestors', 'NNS'), ('many', 'JJ'), ('years', 'NNS'), ('counsellors', 'NNS'), ('syndics', 'VBP'), ('father', 'RB'), ('filled', 'VBN'), ('several', 'JJ'), ('public', 'JJ'), ('situations', 'NNS')]


#### 7. Split the Train and the Test Dataset 

In [32]:
documents = []
for i in range(0,8):
    key = list(chapter_dict.keys())[i]
    append_list = chapter_dict.get(key).split()
    append_tag_list = nltk.pos_tag(append_list)
    documents.append(append_tag_list)

In [33]:
#Getting the tagged sentences
sent_tag = documents
mod_sent_tag=[]
for s in sent_tag:
  s.insert(0,('##','##'))
  s.append(('&&','&&'))
  mod_sent_tag.append(s)

In [34]:
#Splitting the data for train and test
split_num = int(len(mod_sent_tag)*0.9)
train_data = mod_sent_tag[0:split_num]
test_data = mod_sent_tag[split_num:]

#### 8. Implement the Viterbi Algorithm to get the Part of Speech Tagging.

In [35]:
#Creating a dictionary whose keys are tags and values contain words which were assigned the correspoding tag
# ex:- 'TAG':{word1: count(word1,'TAG')}
train_word_tag = {}
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      try:
        train_word_tag[t][w]+=1
      except:
        train_word_tag[t][w]=1
    except:
      train_word_tag[t]={w:1}

In [36]:
#Calculating the emission probabilities using train_word_tag
train_emission_prob={}
for k in train_word_tag.keys():
  train_emission_prob[k]={}
  count = sum(train_word_tag[k].values())
  for k2 in train_word_tag[k].keys():
    train_emission_prob[k][k2]=train_word_tag[k][k2]/count

In [37]:
#Estimating the bigram of tags to be used for transition probability
bigram_tag_data = {}
for s in train_data:
  bi=list(nltk.bigrams(s))
  for b1,b2 in bi:
    try:
      try:
        bigram_tag_data[b1[1]][b2[1]]+=1
      except:
        bigram_tag_data[b1[1]][b2[1]]=1
    except:
      bigram_tag_data[b1[1]]={b2[1]:1}

In [38]:
#Calculating the probabilities of tag bigrams for transition probability  
bigram_tag_prob={}
for k in bigram_tag_data.keys():
  bigram_tag_prob[k]={}
  count=sum(bigram_tag_data[k].values())
  for k2 in bigram_tag_data[k].keys():
    bigram_tag_prob[k][k2]=bigram_tag_data[k][k2]/count

In [39]:
#Calculating the possible tags for each word
#Note: Here we have used the whole data(Train+Test)
#Reason: There may be some words which are not present in train data but are present in test data 
tags_of_tokens = {}
count=0
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l
        
for s in test_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l

In [40]:
#Dividing the test data into test words and test tags
test_words=[]
test_tags=[]
for s in test_data:
  temp_word=[]
  temp_tag=[]
  for (w,t) in s:
    temp_word.append(w.lower())
    temp_tag.append(t)
  test_words.append(temp_word)
  test_tags.append(temp_tag)

In [41]:
#Executing the Viterbi Algorithm
predicted_tags = []                #intializing the predicted tags
for x in range(len(test_words)):   # for each tokenized sentence in the test data
  s = test_words[x]
  #storing_values is a dictionary which stores the required values
  #ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}                
  storing_values = {}              
  for q in range(len(s)):
    step = s[q]
    #for the starting word of the sentence
    if q == 1:                
      storing_values[q] = {}
      tags = tags_of_tokens[step]
      for t in tags:
        #this is applied since we do not know whether the word in the test data is present in train data or not
        try:
          storing_values[q][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][step]]
        #if word is not present in the train data but present in test data we assign a very low probability of 0.0001
        except:
          storing_values[q][t] = ['##',0.0001]#*train_emission_prob[t][step]]
    
    #if the word is not at the start of the sentence
    if q>1:
      storing_values[q] = {}
      previous_states = list(storing_values[q-1].keys())   # loading the previous states
      current_states  = tags_of_tokens[step]               # loading the current states
      #calculation of the best previous state for each current state and then storing
      #it in storing_values
      for t in current_states:                             
        temp = []
        for pt in previous_states:                         
          try:
            temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step])
          except:
            temp.append(storing_values[q-1][pt][1]*0.0001)
        max_temp_index = temp.index(max(temp))
        best_pt = previous_states[max_temp_index]
        storing_values[q][t]=[best_pt,max(temp)]

  #Backtracing to extract the best possible tags for the sentence
  pred_tags = []
  total_steps_num = storing_values.keys()
  last_step_num = max(total_steps_num)
  for bs in range(len(total_steps_num)):
    step_num = last_step_num - bs
    if step_num == last_step_num:
      pred_tags.append('&&')
      pred_tags.append(storing_values[step_num]['&&'][0])
    if step_num<last_step_num and step_num>0:
      pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0])
  predicted_tags.append(list(reversed(pred_tags)))

In [42]:
#Calculating the accuracy based on tagging each word in the test data.
right = 0 
wrong = 0
for i in range(len(test_tags)):
  gt = test_tags[i]
  pred = predicted_tags[i]
  for h in range(len(gt)):
    if gt[h] == pred[h]:
      right = right+1
    else:
      wrong = wrong +1 

#### 9. Calculate the Accuracy and F1 score.

In [43]:
list_set = set(test_words[0])
unique_list = (list(list_set))

In [44]:
len(unique_list)

152

In [45]:
print('Accuracy calculated on the test data is: ',right/(right+wrong))

Accuracy calculated on the test data is:  0.953757225433526


In [46]:
Precision = right/(right+wrong)
recall    = right/len(unique_list)

In [47]:
F1_score = (2*Precision*recall)/(Precision+recall)

###### This above formula is referenced from link: https://www.cl.cam.ac.uk/teaching/1920/MLRD/tasks/task8_copy.pdf

In [48]:
print('F1_score calculated on the test data is: ',F1_score)

F1_score calculated on the test data is:  1.015384615384615


###### F1-score calculated from the formula given along with the questions

In [49]:
print('F1-score on the test data is: ',right/len(unique_list))

F1-score on the test data is:  1.0855263157894737


###### Score calculated for each class from the sklearn's classification report metric

In [50]:
y_test = []
y_pred = []
for i in range(1,len(predicted_tags[0])-1):
    y_pred.append(predicted_tags[0][i])
    y_test.append(test_tags[0][i])

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          CC       1.00      1.00      1.00         1
          CD       1.00      1.00      1.00         3
          DT       1.00      1.00      1.00         4
          IN       1.00      1.00      1.00         3
          JJ       0.92      0.85      0.88        26
          MD       1.00      1.00      1.00         6
          NN       0.96      0.96      0.96        50
         NNS       1.00      1.00      1.00        12
         PRP       1.00      1.00      1.00         7
        PRP$       1.00      1.00      1.00         1
          RB       1.00      1.00      1.00        13
          VB       1.00      1.00      1.00         5
         VBD       0.89      0.94      0.92        18
         VBG       1.00      1.00      1.00         3
         VBN       0.93      0.93      0.93        14
         VBP       0.80      1.00      0.89         4
         WRB       1.00      1.00      1.00         1

    accuracy              

#### 10. Using the LDA algorithm create the Topics (10) for the Corpus  

In [52]:
documents_list = list(chapter_dict.keys())

In [53]:
documents = []
for i in range(0,8):
    documents.append(chapter_dict.get(documents_list[i]))

In [54]:
data_words = [sent.split() for sent in documents]

In [55]:
id2word = corpora.Dictionary(data_words)
texts = data_words
whole_file = [id2word.doc2bow(text) for text in texts]

In [56]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=whole_file,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                      passes = 500)

#### 11. List down the 10 words in each of the Topics Extracted.  

In [57]:
pprint(lda_model.print_topics())

[(0,
  '0.001*"infinite" + 0.001*"how" + 0.001*"limbs" + 0.001*"lifeless" + '
  '0.001*"lay" + 0.001*"infuse" + 0.001*"lustrous" + 0.001*"god" + '
  '0.001*"halfextinguished" + 0.001*"might"'),
 (1,
  '0.001*"infinite" + 0.001*"how" + 0.001*"limbs" + 0.001*"lifeless" + '
  '0.001*"lay" + 0.001*"infuse" + 0.001*"lustrous" + 0.001*"god" + '
  '0.001*"halfextinguished" + 0.001*"might"'),
 (2,
  '0.032*"i" + 0.014*"you" + 0.009*"my" + 0.007*"could" + 0.007*"yet" + '
  '0.007*"she" + 0.007*"elizabeth" + 0.007*"upon" + 0.007*"he" + 0.005*"time"'),
 (3,
  '0.019*"i" + 0.015*"would" + 0.012*"justine" + 0.008*"seemed" + 0.008*"she" '
  '+ 0.008*"a" + 0.008*"yet" + 0.008*"suffered" + 0.008*"cause" + '
  '0.008*"committed"'),
 (4,
  '0.001*"infinite" + 0.001*"how" + 0.001*"limbs" + 0.001*"lifeless" + '
  '0.001*"lay" + 0.001*"infuse" + 0.001*"lustrous" + 0.001*"god" + '
  '0.001*"halfextinguished" + 0.001*"might"'),
 (5,
  '0.048*"i" + 0.007*"it" + 0.007*"great" + 0.005*"in" + 0.005*"pursuit" + '

In [58]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, whole_file, id2word,  mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.120267  0.119835       1        1  35.810638
6      0.121895 -0.138220       2        1  26.066432
2      0.107011  0.148356       3        1  25.260550
3     -0.131795 -0.105354       4        1  12.512886
0      0.003855 -0.004103       5        1   0.058249
1      0.003861 -0.004102       6        1   0.058249
4      0.003865 -0.004102       7        1   0.058249
7      0.003854 -0.004104       8        1   0.058249
8      0.003862 -0.004102       9        1   0.058249
9      0.003859 -0.004103      10        1   0.058249, topic_info=              Term      Freq     Total Category  logprob  loglift
44              he  9.000000  9.000000  Default  30.0000  30.0000
415            you  7.000000  7.000000  Default  29.0000  29.0000
754          would  6.000000  6.000000  Default  28.0000  28.0000
475          great  3.000000  3.000000  Default  27.0000  27.0000
209             it  5.000000  5.000000  Default  26.0000  26.0000
..             ...       ...       ...      ...      ...      ...
25          deeply  0.000849  2.679642  Topic10  -6.8416  -0.6084
26        deplored  0.000849  1.100283  Topic10  -6.8416   0.2817
27     disposition  0.000849  1.889967  Topic10  -6.8416  -0.2593
28   distinguished  0.000849  1.889967  Topic10  -6.8416  -0.2593
29           early  0.000849  1.884623  Topic10  -6.8416  -0.2564

[590 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
416       1  0.410203       a
416       4  0.410203       a
761       1  0.557945  absent
761       4  0.557945  absent
118       2  0.908857     act
...     ...       ...     ...
577       1  0.208726     yet
577       3  0.417451     yet
577       4  0.208726     yet
415       1  0.398639     you
415       3  0.664398     you

[315 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 3, 4, 1, 2, 5, 8, 9, 10])